In [ ]:
# pip install --upgrade scikit-learn

In [51]:
import warnings
from numba import NumbaDeprecationWarning

warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
import joblib
import os
import econml
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor

<h1>1 Imports</h1>

In [52]:
energy_data = pd.read_csv('/home/nissatech/Documents/Double Machine Learning/Raw Data/energy_data.csv')

In [53]:
env_data = pd.read_csv('/home/nissatech/Documents/Double Machine Learning/Raw Data/environment_data.csv')

<h1>2 Indexing</h1>

In [54]:
df_indexed_env_data = env_data.copy()
df_indexed_env_data['timestamp'] = pd.to_datetime(df_indexed_env_data['timestamp'])
df_indexed_env_data.index = df_indexed_env_data['timestamp']
del df_indexed_env_data['timestamp']

In [55]:
df_indexed_energy_data = energy_data.copy()
df_indexed_energy_data['timestamp'] = pd.to_datetime(df_indexed_energy_data['timestamp'])
df_indexed_energy_data.index = df_indexed_energy_data['timestamp']
del df_indexed_energy_data['timestamp']

<h1>3 Dealing with NaN values</h1>

In [56]:
df_indexed_energy_data.isna().any().any()

False

In [57]:
df_indexed_env_data.isna().any().any()

True

In [58]:
num_rows_with_nan = df_indexed_env_data.isna().any(axis=1).sum()
num_rows_with_nan

23

In [59]:
df_indexed_env_data.dropna(inplace=True)

In [60]:
df_indexed_env_data.isna().any().any()

False

In [61]:
df_indexed_energy_data.shape

In [62]:
df_indexed_env_data.shape

<h1>4 Resampling (at a frequency of 1Hz) </h1>

In [63]:
df_resampled_energy = df_indexed_energy_data.resample('1S').mean()
df_interpolated_energy = df_resampled_energy.interpolate()

In [64]:
df_resampled_env = df_indexed_env_data.resample('1S').mean()
df_interpolated_env = df_resampled_env.interpolate()

<h1>5 Index alignment</h1>

In [65]:
# print(df_interpolated_energy.shape)
# print(df_interpolated_energy.index[0])
# print(df_interpolated_energy.index[-1])

In [66]:
# print(df_interpolated_env.shape)
# print(df_interpolated_env.index[0])
# print(df_interpolated_env.index[-1])

In [67]:
# align
df_interpolated_energy = df_interpolated_energy.iloc[:-2]
df_interpolated_env = df_interpolated_env.iloc[2:]

# check indices again
# print(df_interpolated_energy.index[0])
# print(df_interpolated_energy.index[-1])

# print(df_interpolated_env.index[0])
# print(df_interpolated_env.index[-1])

<h1>6 Creating new Dataset for processing</h1>

In [118]:
# Choosing features
columns_to_drop = ['Active Power B average [kW]','Active Power C average [kW]',
                   'Current B average [A]', 'Current C average [A]',
                   'Power Factor B average', 'Power Factor C average',
                   'Reactive Power B average [kVAr]', 'Reactive Power C average [kVAr]', 
                   'THDI B average [%]', 'THDI C average [%]',
                   'THDU B average [%]', 'THDU C average [%]',
                   'Voltage B average [V]', 'Voltage C average [V]',
                   'Active Power A average [kW]', 'Frequency average [Hz]'
                  ]

df_phase_A = df_interpolated_energy.drop(columns=columns_to_drop)

# Define the time range
start_time = pd.Timestamp('2023-04-03 11:35:00')
end_time = pd.Timestamp('2023-04-03 14:30:00')

# Filter the data
mask_two_hours = (df_phase_A.index >= start_time) & (df_phase_A.index < end_time)
df_energy_3 = df_phase_A.loc[mask_two_hours]
df_energy_3 = df_energy_3[df_energy_3['THDI A average [%]'] > 0]

In [119]:
df_env_3 = df_interpolated_env.loc[df_energy_3.index[0]:df_energy_3.index[-1]]

<h1>7 Visualization</h1>

In [ ]:
# for column in df_energy_3.columns:
#     df_energy_3[column].plot(figsize = (14, 8), title = column)
#     plt.show()

In [107]:
# for column in df_env_3.columns:
#     df_env_3[column].plot(figsize = (14, 8), title = column)
#     plt.show()